In [ ]:
import pandas as pd

from Pricing4API.plan import Plan
from Pricing4API.pricing import Pricing


s_second = 1
s_minute= 60
s_hour = 3600
s_day = 3600 * 24
s_month = 3600 * 24 * 30

ListDBLP=[]
ListDBLP.append()

PricingDBLP = Pricing('DBLP', ListDBLP, 'queries')
PricingDBLP.link_plans()

PricingDBLP.create_table()

,0
,Pro
Rate (queries/2s),1
Quota,"[20, 1000]"
Quota Unit,"[1m, 1h]"
Base Cost ($/1s),9.99
Unit Overage Cost ($/queries),None
Max Number of Subscriptions,1


In [4]:
dblp = Plan('Pro', (9.99, 1, None), (1, 2*s_second), [(20, s_minute), (1000, s_hour)])

dblp.compute_t_ast()

[0, 38.0]

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import webbrowser


output_actions = widgets.Output()
output_show_table = widgets.Output()
output_validity = widgets.Output()
output_pricing_bot = widgets.Output()
output_pricing_lab = widgets.Output()
output_csv = widgets.Output()
output_curve = widgets.Output()


# Funciones para cada pestaña

def show_table(b):
    global datasheet
    table = PricingDBLP.create_table()
    
    with output_actions:
        clear_output()
        datasheet = PricingDBLP.show_more_table(table)
        display(datasheet)
        display(button_csv, button_curve)
        
    
    
def export_csv(b):
    with output_actions:
        clear_output()
        table = PricingDBLP.create_table()

        csv_path = 'tabla_dblp.csv'
        table.to_csv(csv_path, index=False)
        webbrowser.open(csv_path)
        display(datasheet)
        print("CSV creado con éxito")

def show_capacity_curve(b):
    with output_actions:
        clear_output(wait=True)
        display(datasheet)
        plans = PricingDBLP.plans
        time_interval=120
        for plan in plans: 
            curve= plan.show_available_capacity_curve(time_interval)
        

def open_main_notebook(b):
    with output_actions:
        clear_output(wait=True)
        display(HTML("<a href='Pricing4ApiLab.ipynb' target='_blank'>Pricing4ApiLab.ipynb</a>"))

def on_button2_clicked(b):
    with output_actions:
        clear_output(wait=True)
        plans = PricingDBLP.plans

        invalid_plans = []
        for i in range(len(plans) - 1):
            current_plan = plans[i]
            next_plan = plans[i + 1]

            if current_plan.price > next_plan.price:
                invalid_plans.append((current_plan.name, next_plan.name))

        if not invalid_plans:
            print("Todos los planes cumplen con la validez de precios.")
        else:
            for prev, nxt in invalid_plans:
                print(f"ADVERTENCIA: El plan {prev} tiene un precio mayor que el plan {nxt}.")

def on_pricing_bot_clicked(b):
    with output_actions:
        clear_output(wait=True)
        history = []

        if 'history' in globals():
            history = globals()['history']

        text = widgets.Text(value='', placeholder='Escribe aquí', description='Input:')
        display(text)

        def handle_submit(sender):
            clear_output(wait=True)
            user_input = text.value
            history.append(("You:", user_input))

            for entry in history:
                print(entry[0], entry[1])

            if user_input == 'Con que pricing estamos trabajando':
                response = "Estamos trabajando con el pricing de " + (PricingDBLP.name) + "."
            elif user_input == 'Cuales son los planes que tiene DBLP':
                response = "DBLP está formado por el plan: " + ", ".join(plan.name for plan in PricingDBLP.plans) + "."
            else:
                response = "No puedo responder a eso."

            history.append(("Bot:", response))
            print("Bot:", response)
            globals()['history'] = history

        text.on_submit(handle_submit)




# Crear botones con sus descripciones
button_show_table = widgets.Button(description="Show Datasheet", tooltip="Show Datasheet")
button_validity = widgets.Button(description="Validity", tooltip="Validity")
button_pricing_bot = widgets.Button(description="Pricing4API Bot", tooltip="Pricing4API Bot")
button_pricing_lab = widgets.Button(description="Pricing4API Lab", tooltip="Pricing4API Lab")
button_csv = widgets.Button(description="Export CSV", tooltip="Export CSV")
button_curve = widgets.Button(description="Capacity Curves", tooltip="Capacity Curves")

# Asignar funciones a los botones
button_validity.on_click(on_button2_clicked)
button_pricing_bot.on_click(on_pricing_bot_clicked)
button_pricing_lab.on_click(open_main_notebook)
button_show_table.on_click(show_table)
button_csv.on_click(export_csv)
button_curve.on_click(show_capacity_curve)

# Organizar botones en pestañas y asignar nombres
tab = widgets.Tab()
tab.children = [button_show_table, button_validity, button_pricing_bot, button_pricing_lab]
tab.set_title(0, "Show Table")
tab.set_title(1, "Validity")
tab.set_title(2, "Pricing4API Bot")
tab.set_title(3, "Pricing4API Lab")


# Mostrar las pestañas como botones con nombres
display(tab, output_show_table, output_actions)

ModuleNotFoundError: No module named 'ipywidgets'